# gsearch_Backend Test Notebook
This notebook provides lightweight assertion-style tests for the Loihi graph-search brick (`LoihiGSBrick`) and its backend (`gsearch_Backend`).


In [9]:
# Imports
import networkx as nx
from fugu.scaffold.scaffold import Scaffold
from fugu.bricks.loihi_gs_brick import LoihiGSBrick
from fugu.backends.gsearch_backend import gsearch_Backend

def build_scaffold(adj, source, destination):
    brick = LoihiGSBrick(adj, source=source, destination=destination, name='GSBrickTest')
    scaffold = Scaffold()
    scaffold.add_brick(brick, output=True)  # no inputs (origin brick)
    scaffold.lay_bricks()
    return scaffold, brick

# Super Dense Random Graph Cost Parity

Demonstrate that the Fugu Loihi graph-search backend achieves the **same shortest-path cost** as a canonical Dijkstra implementation on a very dense random directed graph (with cycles).



Key points:

- We do NOT require identical node-by-node path reconstruction; only cost parity matters.

- Dense cyclic graphs can yield multiple distinct shortest paths of equal cost; backend forward path readout may be ambiguous or empty.

- If backend path reconstruction is empty, we fall back to recomputing the cost along the Dijkstra path to validate the pruning dynamics indirectly.

- Success criterion: `cost_fugu == cost_dijkstra`.



Rationale for ignoring exact path:

Shortest-path algorithms (wavefront pruning vs. heap-based Dijkstra) can legitimately pick different representatives among multiple optimal paths. Cost equivalence is the invariant we care about for correctness.


In [10]:
# Utility: Dijkstra shortest path (returns path, cost)
def dijkstra(adj, src, dst):
    import heapq
    dist = {src: 0}
    prev = {}
    pq = [(0, src)]
    while pq:
        d, u = heapq.heappop(pq)
        if d != dist[u]:
            continue
        if u == dst:
            break
        for v, c in adj.get(u, []):
            nd = d + c
            if nd < dist.get(v, float('inf')):
                dist[v] = nd
                prev[v] = u
                heapq.heappush(pq, (nd, v))
    path = []
    if dst in dist:
        cur = dst
        while cur != src:
            path.append(cur)
            cur = prev[cur]
        path.append(src)
        path.reverse()
    return path, dist.get(dst, float('inf'))

In [11]:
# 4. Super Dense Random Directed Graph Cost Parity

import random

from fugu.bricks.loihi_gs_brick import LoihiGSBrick
from fugu.backends.gsearch_backend import gsearch_Backend
from fugu.scaffold.scaffold import Scaffold

# random.seed(2026)

N = 80            # number of nodes
p = 1          # edge probability for each ordered pair (dense)
max_cost = 64     # edge costs 1..64
src = 0
dst = N - 1

# Build dense directed graph (allow cycles, no self-loops)
adj = {i: [] for i in range(N)}
for i in range(N):
    for j in range(N):
        if i == j:
            continue
        if random.random() < p:
            adj[i].append((j, random.randint(1, max_cost)))

# Dijkstra ground truth
path_d, cost_d = dijkstra(adj, src, dst)
assert cost_d < float('inf'), 'No path found by Dijkstra (unexpected in dense graph).'

# Fugu backend execution
brick = LoihiGSBrick(adj, source=src, destination=dst, name='DenseRandomCostParity')
scaffold = Scaffold()
scaffold.add_brick(brick, output=True)
scaffold.lay_bricks()
backend = gsearch_Backend()
backend.compile(scaffold, {})
result = backend.run(n_steps=5000)
assert result['source_spiked'], 'Source did not spike—wavefront failed.'

# Backend path cost reconstruction (required)
node_map = {v: k for k, v in brick.node_to_neuron.items()}
backend_path_nodes = []
for n in result['path']:
    mapped = node_map.get(n)
    # Filter out auxiliary neurons (they have string names with "__aux__" suffix)
    # We only want actual graph nodes (integers in range [0, N))
    if mapped is not None and isinstance(mapped, int) and 0 <= mapped < N:
        backend_path_nodes.append(mapped)

def edge_cost(u, v):
    return next((c for w, c in adj.get(u, []) if w == v), None)

# Reconstruct cost from backend path
assert backend_path_nodes, 'Backend produced empty path'
assert backend_path_nodes[0] == src and backend_path_nodes[-1] == dst, \
    f'Backend path does not connect source to destination: {backend_path_nodes}'

cost_backend_path = 0
for u, v in zip(backend_path_nodes[:-1], backend_path_nodes[1:]):
    c = edge_cost(u, v)
    assert c is not None, f'Invalid edge in backend path: {u} -> {v}'
    cost_backend_path += c

# Verify cost matches Dijkstra
assert cost_backend_path == cost_d, \
    f'Cost parity failed: backend={cost_backend_path} vs Dijkstra={cost_d}'

# Reporting / diagnostics
print(f"Shortest path cost (Dijkstra): {cost_d}")
print(f"Backend reconstructed path cost: {cost_backend_path}")
print(f"Backend reconstructed path length: {len(backend_path_nodes)}")
print(f"Dijkstra path (first 15 nodes max): {path_d[:15]}{'...' if len(path_d) > 15 else ''}")
print(f"Backend path nodes (first 15 max): {backend_path_nodes[:15]}{'...' if len(backend_path_nodes) > 15 else ''}")
print("SUCCESS: Loihi graph-search pruning produced a shortest-path cost matching Dijkstra.")


Shortest path cost (Dijkstra): 9
Backend reconstructed path cost: 9
Backend reconstructed path length: 4
Dijkstra path (first 15 nodes max): [0, 49, 52, 79]
Backend path nodes (first 15 max): [0, 49, 52, 79]
SUCCESS: Loihi graph-search pruning produced a shortest-path cost matching Dijkstra.


## Divergent Path Sanity Check


Search for a random dense graph where the backend reports a different shortest path than Dijkstra but with the same cost, then verify the backend path is valid and optimal.

In [12]:
# Find a divergent-but-optimal backend path

import random



def find_divergent_optimal(seed_start=0, trials=100, n=60, p=0.8, max_cost=40):

    for seed in range(seed_start, seed_start + trials):

        random.seed(seed)

        adj = {i: [] for i in range(n)}

        for i in range(n):

            for j in range(n):

                if i == j:

                    continue

                if random.random() < p:

                    adj[i].append((j, random.randint(1, max_cost)))



        src, dst = 0, n - 1

        path_d, cost_d = dijkstra(adj, src, dst)

        if cost_d == float('inf'):

            continue  # disconnected sample; skip



        brick = LoihiGSBrick(adj, source=src, destination=dst, name=f'DivergentTrial_{seed}')

        scaffold = Scaffold()

        scaffold.add_brick(brick, output=True)

        scaffold.lay_bricks()



        backend = gsearch_Backend()

        backend.compile(scaffold, {})

        result = backend.run(n_steps=5000)

        if not result['source_spiked']:

            continue  # backend did not finish wavefront



        node_map = {v: k for k, v in brick.node_to_neuron.items()}

        backend_path = []

        for nid in result['path']:

            mapped = node_map.get(nid)

            if isinstance(mapped, int) and 0 <= mapped < n:

                backend_path.append(mapped)



        if len(backend_path) < 2 or backend_path[0] != src or backend_path[-1] != dst:

            continue



        def edge_cost(u, v):

            return next((c for w, c in adj[u] if w == v), None)



        backend_cost = 0

        valid_edges = True

        for u, v in zip(backend_path[:-1], backend_path[1:]):

            w = edge_cost(u, v)

            if w is None:

                valid_edges = False

                break

            backend_cost += w



        if not valid_edges:

            continue



        if backend_cost == cost_d and backend_path != path_d:

            return {

                'seed': seed,

                'adj': adj,

                'dijkstra_path': path_d,

                'backend_path': backend_path,

                'cost': backend_cost,

            }



    return None





divergent_example = find_divergent_optimal()

if divergent_example is None:

    print("No divergent-but-optimal example found in the scanned seeds; try increasing trials or adjusting parameters.")

else:

    print(f"Seed: {divergent_example['seed']}")

    print(f"Shortest-path cost (shared): {divergent_example['cost']}")

    print(f"Dijkstra path length: {len(divergent_example['dijkstra_path'])} -> {divergent_example['dijkstra_path']}")

    print(f"Backend path length: {len(divergent_example['backend_path'])} -> {divergent_example['backend_path']}")



    # Optional: double-check no cheaper path exists by re-running Dijkstra on backend path nodes

    # (cost equality already implies optimality, but we recompute to display the edge weights). 

    weights = []

    for u, v in zip(divergent_example['backend_path'][:-1], divergent_example['backend_path'][1:]):

        w = next((c for w_node, c in divergent_example['adj'][u] if w_node == v))

        weights.append(w)

    print(f"Backend edge weights: {weights}")

    print(f"Sum of backend edge weights: {sum(weights)}")

Seed: 12
Shortest-path cost (shared): 9
Dijkstra path length: 7 -> [0, 6, 46, 31, 53, 18, 59]
Backend path length: 5 -> [0, 48, 53, 18, 59]
Backend edge weights: [4, 3, 1, 1]
Sum of backend edge weights: 9
